In [9]:
import pandas as pd
import numpy as np
import re


In [10]:
def import_and_concat(file, sheet_1, sheet_2):

    df1 = pd.read_excel(file, sheet_name = sheet_1)
    display(df1.shape)
    df2 = pd.read_excel(file, sheet_name = sheet_2)
    display(df2.shape)
    df = pd.concat([df1, df2], axis = 0, ignore_index= True)
    display(df.shape)
    return df

df = import_and_concat('Data/online_retail_II.xlsx', 'Year 2009-2010', 'Year 2010-2011')

(525461, 8)

(541910, 8)

(1067371, 8)

### Data Cleaning

Throughout working with this dataset, several rows appear to be from testing processes, price adjustments, errors and missing data. The following block of code cleans out these rows from the dataset, and saves the cleaned data in the repository. 

In [3]:
def drop_rows(mask):                           #Establishing a function to drop rows based on a dataframe filter.
    df.drop(mask.index, inplace = True)        #There are many rows to drop going forward, so this will eliminate code redundancies.

drop_rows(df.loc[(df["Description"] == "This is a test product.") | (df.StockCode == "ADJUST")])
drop_rows(df[df.Description.str.contains("wrongly", na = False)])
drop_rows(df.loc[df.StockCode == "ADJUST"]) 
drop_rows(df.loc[df.StockCode == "B"])

#Correcting country name to match general country naming convention. 

df["Country"].replace({"EIRE": "Ireland", "USA": "United States of America", "RSA": "South Africa"}, inplace = True)
#Correcting a typo in item description.
df["Description"].replace({"PACK OF 72 RETRO SPOT CAKE CASES": "PACK OF 72 RETROSPOT CAKE CASES"}, inplace = True)

#Creating a revenue feature.
df["Revenue"] = df["Quantity"] * df["Price"]

df.to_csv("Data/cleaned_total_data", index = False) 

### Shipping-related Data

The following extracts rows which are not related to products, most of which is shipping-related expenses. 

In [4]:
shipping_df = df.copy()
shipping_df["StockCode"] = df["StockCode"].str.extract('(^\D+)', expand = False)

shipping_df.dropna(subset = ["StockCode"], inplace = True)  #Dropping rows without shipping information 
shipping_df.to_csv('Data/cleaned_shipping_data', index = False) #Saving this dataframe for future analysis

display(shipping_df.head())
display(shipping_df.StockCode.value_counts())

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
89,489439,POST,POSTAGE,3,2009-12-01 09:28:00,18.00,12682.0,France,54.00
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.00,12636.0,United States of America,141.00
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.00,12362.0,Belgium,130.00
625,489526,POST,POSTAGE,6,2009-12-01 11:50:00,18.00,12533.0,Germany,108.00
735,C489535,D,Discount,-1,2009-12-01 12:11:00,9.00,15299.0,United Kingdom,-9.00
...,...,...,...,...,...,...,...,...,...
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,-224.69
1067191,581570,POST,POSTAGE,1,2011-12-09 11:59:00,18.00,12662.0,Germany,18.00
1067228,581574,POST,POSTAGE,2,2011-12-09 12:09:00,18.00,12526.0,Germany,36.00
1067229,581578,POST,POSTAGE,3,2011-12-09 12:16:00,18.00,12713.0,Germany,54.00


In [6]:
df.drop(shipping_df.index, inplace = True, axis = 0)                  #Dropping the shipping data from the main dataset

### Refunds-related Data

The following extracts refunds-related data from the main dataset for ease of sales analysis in other workbooks.

In [7]:
df["refund"] = df['Invoice'].str.extract("(^\D+[0-9]+)")      #Identifying the rows which are returns by the invoice number
drop_rows(df.loc[df.refund.notnull()])                        #Dropping these rows from the main dataset
df.Invoice = df.Invoice.astype(int)                           #Checking they were all dropped by turing the Invoice feature into an int. 

df['StockCode'].replace('\D+', '', regex=True, inplace = True)   #Removing any trailing letters from the Stockcode of each product
drop_rows(df[df["Quantity"]<=0])                               #Dropping rows where quantity is less than 0.

df.to_csv("Data/cleaned_sales_data", index = False)            #Fanally, saving a copy of the cleaned dataset. 